Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-multi-model/multi-model-register-and-deploy.png)

# Deploy Multiple Models as Webservice

This example shows how to deploy a Webservice with multiple models in step-by-step fashion:

 1. Register Models
 2. Deploy Models as Webservice

## Prerequisites
If you are using an Azure Machine Learning Compute Instance Notebook, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Register Models

In this example, we will be using and registering x number of models

First we will train those simple models on the [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) included with scikit-learn, serializing them to files in the current directory.


In [ ]:
import joblib
import sklearn
import os

from sklearn.datasets import load_diabetes
from sklearn.linear_model import BayesianRidge, Ridge, LogisticRegression

path = "./models/"
counter = 100

if not os.path.exists(path):
    os.makedirs(path, exist_ok=True)

x, y = load_diabetes(return_X_y=True)

# loop on the training using ridge (with alpha varying between 1 and 1+(counter/100)) and dumping phase 

for a in range(1, counter, 1):

    model = Ridge(alpha=1+a/100).fit(x, y)
    model_path = str(path) + "mml_model_" + str(a) +".pkl"
    joblib.dump(model, model_path)

print("Trained models using scikit-learn {}.".format(sklearn.__version__))

Now that we have our trained models locally, we will register them as Models in the workspace.
adding tags to recognize them

In [ ]:
from azureml.core.model import Model

models_name_selection = []
models_selection = []

# loop on the registration of the models and creating two lists to be used in the following steps. 

for i in range(1, counter, 1):
    
    model_path = str(path) + "mml_model_" + str(i) +".pkl"
    model_name = "mml_model_" + str(i)
    models_name_selection.append(model_name)
    
    model_name = Model.register(model_path=model_path,
               model_name=model_name,
               workspace=ws,
               tags={'mml': 'true'})
    models_selection.insert(i, model_name)



Printing the lists :


In [ ]:
# print(models_name_selection)
# print(*models_selection, sep = "\n")

## Write the Entry Script
Write the script that will be used to predict on your models

In [ ]:
%%writefile score.py

import joblib
import json
import numpy as np

from azureml.core.model import Model

def init():

    # initiate a list to contains models
    # this list will be be automatically populated in the next cell
    
    models_ws = modellisttobeupdated
    
    global models
    models = {}
    i = 0
    
    # loop on the model name list in order to load each model 
    # and insert it into a global models dictionary used in the run function
    
    for mod in models_ws:
        
        model_path = Model.get_model_path(model_name=mod)
        model = joblib.load(model_path)
        models[mod]=model
        
def run(raw_data):
    try:
        
        #get the model_id from the raw data
        model_id = str(json.loads(raw_data)['model_id'])        
        
        #get the data input from the raw data for the model inference
        data = json.loads(raw_data)['data']
        data = np.array(data)
        
        #infer the model identified and get results
        result = models[model_id].predict(data)
 
        return {"Prediction": result.tolist()}
        
    except Exception as e:
        
        result = str(e)
        return result
    


Replace modellisttobeupdated with the correct models name list


In [ ]:
#read input file
fin = open("score.py", "rt")
#read file contents to string
data = fin.read()
#replace all occurrences of the required string
data = data.replace('modellisttobeupdated', str(models_name_selection))
#close the input file
fin.close()
#open the input file in write mode
fin = open("score.py", "wt")
#override the input file with the resulting data
fin.write(data)
#close the file
fin.close()

## Create Environment

You can now create and/or use an Environment object when deploying a Webservice. The Environment can have been previously registered with your Workspace, or it will be registered with it as a part of the Webservice deployment. Please note that your environment must include azureml-defaults with version >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

More information can be found in our [using environments notebook](../training/using-environments/using-environments.ipynb).

In [ ]:
from azureml.core import Environment

env = Environment("deploytocloudenv")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

### Deploy Model as Webservice on Azure Container Instance

Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-multimodel-dep"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, models_selection, inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)


#### Test web service

In [ ]:
import json

model_id_value = 'mml_model_99'
test_sample = json.dumps({'model_id': model_id_value ,'data': x[0:2].tolist()})

prediction = service.run(test_sample)

print(prediction)

#### Delete ACI to clean up

In [ ]:
service.delete()